# FogNet: Process Group-wise SHAP Values

https://gridftp.tamucc.edu/fognet/

- This notebook is part of the [FogNet model repository](https://github.com/conrad-blucher-institute/FogNet)
- Calculates SHAP values for each of the five physics-based channel groups.
- Two variants of SHAP are included: `SHAP` and `LossSHAP`.
- Uses pre-computed FogNet model outputs.

**`SHAP`**, _local feature effect_:
- The SHAP values are the contribution of each group toward a single model outcome.
- So there is are 5 SHAP values for each sample.

**`LossSHAP`**, _global feature importance_:
- The SHAP values are the contribution of each group toward overall model performance.
- So there is a single SHAP value for each group.



## Setup

In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import json
import pickle
import random
import os
from itertools import combinations
from math import comb
from numpy import savez_compressed
from optparse import OptionParser
from scipy.interpolate import (UnivariateSpline, RectBivariateSpline, RegularGridInterpolator)
from sklearn.metrics import confusion_matrix

In [2]:
# Remote input files
group_shap_values = "https://github.com/conrad-blucher-institute/FogNet/raw/main/inference/XAI/group-based-xai/SHAP.zip"
outcomes_file_remote = "https://raw.githubusercontent.com/conrad-blucher-institute/FogNet/main/inference/trained_model/test_outcomes.csv"
outputs_file_remote = "https://raw.githubusercontent.com/conrad-blucher-institute/FogNet/main/inference/trained_model/test_preds.csv"

# Local files
csv_files_dir = "SHAP/"
outcomes_file = "test_outcomes.csv"
outputs_file = "test_preds.csv"

# Output files
output_SHAP_file = "xai_groupwise_shap.csv"
output_LossSHAP_file = "xai_groupwise_lossshap.csv"

In [3]:
!rm $outcomes_file
!rm $outputs_file
!rm $targets_file

# Download & extract data
!wget $group_shap_values -O SHAP.zip -q
!unzip -o SHAP.zip -q
!wget $outcomes_file_remote -q
!wget $outputs_file_remote -q
!wget $targets_file_remote -q

rm: missing operand
Try 'rm --help' for more information.
Archive:  SHAP.zip
caution: filename not matched:  -q
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [4]:
# Load model predictions & classification outcomes
dfOutcomes = pd.read_csv(outcomes_file)
outcome_types = dfOutcomes["outcome"].unique()
dfOutputs = pd.read_csv(outputs_file)
dfOutcomes["pred_fog"] = dfOutputs['pred_fog']

## Calculations used for `SHAP` and `LossSHAP`

In [5]:
# set the number of SHAP players (here, number of groups)
NumPlayers = 5    

# Load all holdout data into a dictionary of dataframes
holdout_results = {}
#  For file Gx..Gy   that means that Gx..Gy are left alone and all other groups have their values randomized between 0-1
for file in os.listdir(csv_files_dir):
    holdout_results[file] = pd.read_csv(os.path.join(csv_files_dir, file, 'VIS_Prob_TEST.csv'))

# Filename with no holdouts is G60, but the others are named by their membership so change to G1G2G3G4G5
holdout_results['G1G2G3G4G5']=holdout_results['G60']
del holdout_results['G60']

# Generate all combinations of players (groups)
combinations_of_groups = []
for s in range(1,NumPlayers+1):
    for i in combinations(range(1,NumPlayers+1), s):
        combinations_of_groups.append(i) 

## SHAP

In [6]:
# Base case: mean model output
base_case = holdout_results['G1G2G3G4G5']['C0_Prob'].mean() 

def calc_SHAP(teams_without_player_i, i):
    # Start with empty teams (base case)
    b = 'Base'
    a = f'G{i}'
    w = 1/(NumPlayers)

    SHAP_val = w*(holdout_results[a]['C0_Prob'] - base_case)

    # Init count array
    cc = np.zeros(NumPlayers, dtype=int)  
    for team in teams_without_player_i:
        # count the number for each size (of group)
        cc[len(team)] = cc[len(team)] + 1 
 
    for team in teams_without_player_i:
        team_with_i = list(team)
        team_with_i.append(i)
        team_with_i.sort()
        # Get team names.  This is to find where the results are stored
        b = ''.join([f'G{s}' for s in team ])
        a = ''.join([f'G{s}' for s in team_with_i ])
        w = 1/(NumPlayers*cc[len(team)])

        # Calc the SHAP value
        SHAP_val = SHAP_val + w*(holdout_results[a]['C0_Prob']-holdout_results[b]['C0_Prob'])
    
    return (SHAP_val)
   
dfOutcomes_ = dfOutcomes.copy()
all_outcomes = dfOutcomes["outcome"].unique().tolist()
for i in range(1,NumPlayers+1):
    v2 = [s for s in combinations_of_groups if i not in s]
    dfOutcomes_[f"G{i}"] = calc_SHAP(v2, i)
    
dfOutcomes = dfOutcomes_

In [7]:
dfOutcomes

,pred_fog,pred_non,pred_class,pred_className,target_class,target_className,outcome,outcome_name,year,G1,G2,G3,G4,G5
0,1.640886e-06,0.999998,1,non-fog,1,non-fog,d,correct-reject,2018,-0.009675,-0.009680,-0.009680,-0.009678,-0.009679
1,2.718333e-05,0.999973,1,non-fog,1,non-fog,d,correct-reject,2018,-0.009653,-0.009645,-0.009781,-0.009645,-0.009641
2,4.197544e-07,1.000000,1,non-fog,1,non-fog,d,correct-reject,2018,-0.009677,-0.009678,-0.009683,-0.009678,-0.009676
3,7.129707e-10,1.000000,1,non-fog,1,non-fog,d,correct-reject,2018,-0.009679,-0.009679,-0.009678,-0.009679,-0.009679
4,2.935946e-06,0.999997,1,non-fog,1,non-fog,d,correct-reject,2018,-0.009679,-0.009679,-0.009678,-0.009677,-0.009678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2223,2.203034e-06,0.999998,1,non-fog,1,non-fog,d,correct-reject,2020,-0.009678,-0.009678,-0.009678,-0.009678,-0.009678
2224,3.077575e-05,0.999969,1,non-fog,1,non-fog,d,correct-reject,2020,-0.009644,-0.009656,-0.009654,-0.009708,-0.009702
2225,1.429611e-06,0.999999,1,non-fog,1,non-fog,d,correct-reject,2020,-0.009680,-0.009692,-0.009713,-0.009650,-0.009657
2226,4.983135e-09,1.000000,1,non-fog,1,non-fog,d,correct-reject,2020,-0.009678,-0.009679,-0.009679,-0.009678,-0.009679


In [8]:
# Write
dfOutcomes.to_csv(output_SHAP_file, index=False)

## LossSHAP

In [9]:
# Optimization threshold
thresh = 0.193

In [10]:
# Binarize predictions using threshold
def pred2class(preds, thresh = 0.193):
  classes = np.array(preds > thresh).astype("int")
  return classes

In [11]:
# Calculate model performance metrics
def calcScores(y_true, y_pred):
  hit, miss, false_alarm, correct_reject = confusion_matrix(y_true, y_pred).ravel()
  
  POD  = hit / (hit + miss)
  F    = false_alarm / (false_alarm + correct_reject)
  FAR  = false_alarm / (hit + false_alarm)
  CSI  = hit / (hit + false_alarm + miss)
  PSS  = ((hit * correct_reject) - (false_alarm * miss)) / ((false_alarm + correct_reject) * (hit + miss))
  HSS  = (2 * ((hit * correct_reject) - (false_alarm * miss))) / (((hit + miss) * (miss + correct_reject)) + ((hit + false_alarm) * (false_alarm + correct_reject)))
  ORSS = ((hit * correct_reject) - (false_alarm * miss)) / ((hit * correct_reject) + (false_alarm * miss))
  CSS  = ((hit * correct_reject) - (false_alarm * miss)) / ((hit + false_alarm) * (miss + correct_reject))

  scores = {
      "hit"            : hit,
      "miss"           : miss,
      "false_alarm"    : false_alarm,
      "correct_reject" : correct_reject,
      "POD"            : POD,
      "F"              : F,
      "FAR"            : FAR,
      "CSI"            : CSI,
      "PSS"            : PSS,
      "HSS"            : HSS,
      "ORSS"           : ORSS,
      "CSS"            : CSS,
  }

  return scores

In [12]:
# Base case: model metric
y_true = dfOutcomes["target_class"]
y_pred = dfOutcomes["pred_class"]

base_case = calcScores(y_true, y_pred)

In [13]:
def calc_LossSHAP(teams_without_player_i, i, metric="PSS"):
    # Start with empty teams (base case)
    a = f'G{i}'
    w = 1/(NumPlayers)

    SHAP_val = w*(calcScores(y_true, pred2class(holdout_results[a]['C1_Prob'], thresh))[metric] - 0) #base_case[metric])

    # Init count array
    cc = np.zeros(NumPlayers, dtype=int)  
    for team in teams_without_player_i:
        # count the number for each size (of group)
        cc[len(team)] = cc[len(team)] + 1 
 
    for team in teams_without_player_i:
        team_with_i = list(team)
        team_with_i.append(i)
        team_with_i.sort()
        # Get team names.  This is to find where the results are stored
        b = ''.join([f'G{s}' for s in team ])
        a = ''.join([f'G{s}' for s in team_with_i ])
        w = 1/(NumPlayers*cc[len(team)])

        # Calc the SHAP value
        SHAP_val = SHAP_val + w*(
            calcScores(y_true, pred2class(holdout_results[a]['C1_Prob'], thresh))[metric] - \
            calcScores(y_true, pred2class(holdout_results[b]['C1_Prob'], thresh))[metric]
            )
    
    return (SHAP_val)
   
dfOutcomes_ = dfOutcomes.copy()
all_outcomes = dfOutcomes["outcome"].unique().tolist()

dfLossShap = pd.DataFrame(
    columns=["Group"],
)
dfLossShap["Group"] = ["G1", "G2", "G3", "G4", "G5"]

for metric in ["PSS", "HSS", "CSI"]:
  values = np.zeros(NumPlayers)
  for i in range(1,NumPlayers+1):
      v2 = [s for s in combinations_of_groups if i not in s]
      values[i-1] = calc_LossSHAP(v2, i, metric = metric);

  dfLossShap[metric] = values

<ipython-input-11-6348ecea252e>:7: RuntimeWarning: invalid value encountered in long_scalars
  FAR  = false_alarm / (hit + false_alarm)
<ipython-input-11-6348ecea252e>:11: RuntimeWarning: invalid value encountered in long_scalars
  ORSS = ((hit * correct_reject) - (false_alarm * miss)) / ((hit * correct_reject) + (false_alarm * miss))
<ipython-input-11-6348ecea252e>:12: RuntimeWarning: invalid value encountered in long_scalars
  CSS  = ((hit * correct_reject) - (false_alarm * miss)) / ((hit + false_alarm) * (miss + correct_reject))
<ipython-input-11-6348ecea252e>:7: RuntimeWarning: invalid value encountered in long_scalars
  FAR  = false_alarm / (hit + false_alarm)
<ipython-input-11-6348ecea252e>:11: RuntimeWarning: invalid value encountered in long_scalars
  ORSS = ((hit * correct_reject) - (false_alarm * miss)) / ((hit * correct_reject) + (false_alarm * miss))
<ipython-input-11-6348ecea252e>:12: RuntimeWarning: invalid value encountered in long_scalars
  CSS  = ((hit * correct_reject

In [14]:
dfLossShap

,Group,PSS,HSS,CSI
0,G1,0.118632,0.122124,0.083189
1,G2,0.064523,0.053388,0.039151
2,G3,0.065892,0.035456,0.032234
3,G4,0.217257,0.259425,0.170030
4,G5,0.069739,0.046945,0.038139


In [15]:
base_case

{'hit': 37,
 'miss': 30,
 'false_alarm': 35,
 'correct_reject': 2126,
 'POD': 0.5522388059701493,
 'F': 0.016196205460434984,
 'FAR': 0.4861111111111111,
 'CSI': 0.3627450980392157,
 'PSS': 0.5360426005097143,
 'HSS': 0.517337457172948,
 'ORSS': 0.973655158570855,
 'CSS': 0.49997423211708925}

In [16]:
# Sum up SHAP values
dfLossShap.sum()

Group    G1G2G3G4G5
PSS        0.536043
HSS        0.517337
CSI        0.362745
dtype: object

In [17]:
# LossShap value as percentage of metric
for metric in ["PSS", "HSS", "CSI"]:
  dfLossShap[metric + "_relative"] = dfLossShap[metric] / base_case[metric]

dfLossShap

,Group,PSS,HSS,CSI,PSS_relative,HSS_relative,CSI_relative
0,G1,0.118632,0.122124,0.083189,0.221311,0.236062,0.229333
1,G2,0.064523,0.053388,0.039151,0.120368,0.103198,0.107931
2,G3,0.065892,0.035456,0.032234,0.122923,0.068536,0.088862
3,G4,0.217257,0.259425,0.170030,0.405299,0.501461,0.468732
4,G5,0.069739,0.046945,0.038139,0.130100,0.090743,0.105141


In [18]:
# Write
dfLossShap.to_csv(output_LossSHAP_file, index=False)